In [1]:
#importing the relevant libraries
import pandas as pd
import numpy as np
import os
from glob import glob
import sys
pd.set_option('display.max_columns', None, 'display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
data_wheather = pd.read_csv('./cleaned/wheather.CSV',decimal=",")
data_wheather['DATE_ONLY'] = pd.to_datetime(data_wheather['TIME_UTC_STR']).dt.strftime('%m/%d/%Y')
data_wheather = data_wheather.sort_values(by=['DATE_ONLY']).drop(columns=["Unnamed: 8"])

data_wheather = data_wheather.assign(season_year = lambda x: x['file_loc'].str.split('\\').str[1])\
                .assign(location = lambda x: x['file_loc'].str.split('\\').str[2])\
                .assign(match_type = lambda x: x['file_loc'].str.split('\\').str[3])\
                .assign(match_name = lambda x: x['file_loc'].str.split('\\').str[4])\
                .assign(match_name = lambda x: x['match_name'].str.split('_').str[2])\
                .assign(match_name = lambda x: x['match_name'].str.split('.').str[0])\
                .drop(columns = ['file_loc'])

In [3]:
aggregation = {'AIR_TEMP':[('mean_air_temp','mean'),
                ('std_air_temp','std'),
                ('med_air_temp','median'),
                ('max_air_temp','max'),
                ('min_air_temp','min'),
                ('range_air_temp',lambda x: max(x) - min(x)),
                ('skew_air_temp','skew')],
              'TRACK_TEMP' : [('mean_track_temp','mean'),
                ('std_track_temp','std'),
                ('med_track_temp','median'),
                ('max_track_temp','max'),
                ('min_track_temp','min'),
                ('range_track_temp',lambda x: max(x) - min(x)),
                ('skew_track_temp','skew')],
                'HUMIDITY' : [('mean_HUMIDITY','mean'),
                            ('std_HUMIDITY','std'),
                            ('med_HUMIDITY','median'),
                            ('max_HUMIDITY','max'),
                            ('min_HUMIDITY','min'),
                            ('range_HUMIDITY',lambda x: max(x) - min(x)),
                            ('skew_HUMIDITY','skew')],
                'PRESSURE' : [('mean_PRESSURE','mean'),
                    ('std_PRESSURE','std'),
                    ('med_PRESSURE','median'),
                    ('max_PRESSURE','max'),
                    ('min_PRESSURE','min'),
                    ('range_PRESSURE',lambda x: max(x) - min(x)),
                    ('skew_PRESSURE','skew')],

            'WIND_SPEED' : [('mean_WIND_SPEED','mean'),
                          ('std_WIND_SPEED','std'),
                          ('med_WIND_SPEED','median'),
                          ('max_WIND_SPEED','max'),
                          ('min_WIND_SPEED','min'),
                          ('range_WIND_SPEED',lambda x: max(x) - min(x)),
                          ('skew_WIND_SPEED','skew'),
                        ]
              }

#data_wheather[aggregation.keys] = data_wheather[aggregation.keys()].apply(pd.to_numeric)

data_wheather['match_key'] = data_wheather['DATE_ONLY'] + data_wheather['match_name']
df = data_wheather.groupby(['match_key','DATE_ONLY','location','season_year','match_name','match_type'], as_index=True).agg(aggregation).reset_index()
df.columns = df.columns.droplevel()
column_names = df.columns.values
column_names[:6] = ['match_key','DATE_ONLY','location','season_year','match_name','match_type']
df.columns = column_names
df = df.drop(columns = ['match_key'])
#df.to_csv('./cleaned/weather_processed.csv',index = False)

In [8]:
df.shape

(175, 40)

In [4]:
df.head()

,DATE_ONLY,location,season_year,match_name,match_type,mean_air_temp,std_air_temp,med_air_temp,max_air_temp,min_air_temp,range_air_temp,skew_air_temp,mean_track_temp,std_track_temp,med_track_temp,max_track_temp,min_track_temp,range_track_temp,skew_track_temp,mean_HUMIDITY,std_HUMIDITY,med_HUMIDITY,max_HUMIDITY,min_HUMIDITY,range_HUMIDITY,skew_HUMIDITY,mean_PRESSURE,std_PRESSURE,med_PRESSURE,max_PRESSURE,min_PRESSURE,range_PRESSURE,skew_PRESSURE,mean_WIND_SPEED,std_WIND_SPEED,med_WIND_SPEED,max_WIND_SPEED,min_WIND_SPEED,range_WIND_SPEED,skew_WIND_SPEED
0,01/04/2017,Mexico,2016-2017,Groups Qualifying Session,Q,27.101836,0.574968,27.4444,27.7222,26.3889,1.3333,-0.406788,21.574100,0.209570,21.6667,21.6667,21.1111,0.5556,-1.855802,31.142857,0.354169,31.0,32,31,1,2.117634,779.436476,0.198200,779.4985,779.719,779.143,0.576,-0.184513,7.165413,1.516478,8.04672,8.04672,3.21869,4.82803,-1.417020
1,01/04/2017,Mexico,2016-2017,Non Qualifying Practice 1,FP1,16.888880,1.024915,17.3333,19.2222,15.0556,4.1666,-0.614885,21.913584,0.816790,21.6667,23.3333,20.5556,2.7777,0.158910,55.155556,0.366529,55.0,56,55,1,1.966912,780.824667,0.061080,780.8020,780.938,780.734,0.204,0.158626,2.610713,1.690992,3.21869,4.82803,0.00000,4.82803,-0.154161
2,01/04/2017,Mexico,2016-2017,Non Qualifying Practice 2,FP2,25.546313,0.219368,25.6667,25.7778,25.1667,0.6111,-0.918498,20.555600,0.000000,20.5556,20.5556,20.5556,0.0000,0.000000,33.266667,0.449776,33.0,34,33,1,1.111663,780.391367,0.062756,780.3960,780.531,780.260,0.271,0.475190,5.954573,2.902056,6.43738,14.48410,1.60934,12.87476,0.898993
3,01/04/2017,Mexico,2016-2017,Race,Race,27.262177,0.175923,27.2778,27.5000,26.9444,0.5556,-0.645954,23.245589,0.204359,23.3333,23.3333,22.7778,0.5555,-1.927487,34.368421,0.486664,34.0,35,34,1,0.560401,777.901351,0.120517,777.8900,778.229,777.687,0.542,0.643746,7.482039,3.310649,6.43738,19.31210,3.21869,16.09341,1.271079
4,01/04/2017,Mexico,2016-2017,Super Pole Session,Q,27.817464,0.145223,27.8889,27.9444,27.5556,0.3888,-1.383638,22.222200,0.000000,22.2222,22.2222,22.2222,0.0000,0.000000,30.642857,0.497245,31.0,31,30,1,-0.670360,778.876857,0.060905,778.8890,778.974,778.770,0.204,-0.435894,6.782234,3.167426,6.43738,16.09340,3.21869,12.87471,1.983345


In [6]:
#drop seasong year
#make changes to DATE_ONLY to extract year 

str

03/03/2018    8
10/06/2018    8
04/28/2018    8
04/14/2018    8
03/02/2018    8
10/06/2017    8
03/17/2018    8
07/15/2017    8
07/30/2017    8
11/06/2017    8
01/13/2018    8
05/20/2017    8
05/13/2017    8
05/19/2018    8
07/14/2018    8
07/15/2018    7
07/16/2017    7
07/29/2017    7
03/12/2017    7
02/12/2017    7
12/11/2016    5
01/04/2017    5
02/18/2017    5
09/10/2016    4
01/12/2017    1
Name: DATE_ONLY, dtype: int64